In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics       import mean_absolute_error, mean_squared_error
# import plotly.plotly as py
# import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
# from tensorflow import set_random_seed
from numpy.random import seed
# set_random_seed(1)
seed(1)

### Loading data

In [2]:
train = pd.read_csv('./data/train.csv', parse_dates=['date'])
test = pd.read_csv('./data/test.csv', parse_dates=['date'])

### Train set

In [3]:
train.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [4]:
train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


### Time period of the train dataset

In [5]:
print('Min date from train set: %s' % train['date'].min().date())
print('Max date from train set: %s' % train['date'].max().date())

Min date from train set: 2013-01-01
Max date from train set: 2017-12-31


In [6]:
lag_size = (test['date'].max().date() - train['date'].max().date()).days
print('Max date from train set: %s' % train['date'].max().date())
print('Max date from test set: %s' % test['date'].max().date())
print('Forecast lag size', lag_size)

Max date from train set: 2017-12-31
Max date from test set: 2023-01-01
Forecast lag size 1827


In [11]:
train = train[(train['date'] >= '2017-01-01')]

### Rearrange dataset so we can apply shift methods

In [12]:
train_gp = train.sort_values('date').groupby(['item', 'store', 'date'], as_index=False)
train_gp = train_gp.agg({'sales':['mean']})
train_gp.columns = ['item', 'store', 'date', 'sales']
train_gp.head()

,item,store,date,sales
0,1,1,2017-01-01,19.0
1,1,1,2017-01-02,15.0
2,1,1,2017-01-03,10.0
3,1,1,2017-01-04,16.0
4,1,1,2017-01-05,14.0


### Transform the data into a time series problem

In [13]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

#### We will use the current timestep and the last 29 to forecast 90 days ahead

In [14]:
window = 29
lag = lag_size
series = series_to_supervised(train_gp.drop('date', axis=1), window=window, lag=lag)
series.head()

,item(t-29),store(t-29),sales(t-29),item(t-28),store(t-28),sales(t-28),item(t-27),store(t-27),sales(t-27),item(t-26),...,sales(t-2),item(t-1),store(t-1),sales(t-1),item(t),store(t),sales(t),item(t+1827),store(t+1827),sales(t+1827)
29,1.0,1.0,19.0,1.0,1.0,15.0,1.0,1.0,10.0,1.0,...,16.0,1.0,1.0,24.0,1,1,9.0,1.0,6.0,14.0
30,1.0,1.0,15.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,...,24.0,1.0,1.0,9.0,1,1,17.0,1.0,6.0,9.0
31,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,14.0,1.0,...,9.0,1.0,1.0,17.0,1,1,15.0,1.0,6.0,14.0
32,1.0,1.0,16.0,1.0,1.0,14.0,1.0,1.0,24.0,1.0,...,17.0,1.0,1.0,15.0,1,1,17.0,1.0,6.0,14.0
33,1.0,1.0,14.0,1.0,1.0,24.0,1.0,1.0,14.0,1.0,...,15.0,1.0,1.0,17.0,1,1,24.0,1.0,6.0,16.0


#### Drop rows with different item or store values than the shifted columns

In [15]:
last_item = 'item(t-%d)' % window
last_store = 'store(t-%d)' % window
series = series[(series['store(t)'] == series[last_store])]
series = series[(series['item(t)'] == series[last_item])]

#### Remove unwanted columns

In [16]:
columns_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['item', 'store']]
for i in range(window, 0, -1):
    columns_to_drop += [('%s(t-%d)' % (col, i)) for col in ['item', 'store']]
series.drop(columns_to_drop, axis=1, inplace=True)
series.drop(['item(t)', 'store(t)'], axis=1, inplace=True)

### Train/validation split

In [17]:
# Label
labels_col = 'sales(t+%d)' % lag_size
labels = series[labels_col]
series = series.drop(labels_col, axis=1)

X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.4, random_state=0)
print('Train set shape', X_train.shape)
print('Validation set shape', X_valid.shape)
X_train.head()

Train set shape (99790, 30)
Validation set shape (66528, 30)


,sales(t-29),sales(t-28),sales(t-27),sales(t-26),sales(t-25),sales(t-24),sales(t-23),sales(t-22),sales(t-21),sales(t-20),...,sales(t-9),sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t)
85818,54.0,26.0,34.0,33.0,33.0,51.0,31.0,47.0,23.0,33.0,...,37.0,52.0,33.0,41.0,26.0,38.0,44.0,41.0,52.0,25.0
92956,63.0,52.0,37.0,57.0,43.0,49.0,53.0,59.0,38.0,43.0,...,51.0,56.0,37.0,36.0,36.0,50.0,42.0,52.0,50.0,31.0
94730,71.0,76.0,86.0,64.0,72.0,55.0,70.0,85.0,76.0,101.0,...,78.0,92.0,82.0,91.0,64.0,68.0,62.0,73.0,84.0,102.0
118758,61.0,72.0,48.0,45.0,51.0,65.0,69.0,90.0,76.0,44.0,...,96.0,70.0,78.0,44.0,47.0,59.0,66.0,69.0,84.0,72.0
111080,72.0,39.0,38.0,32.0,58.0,53.0,60.0,68.0,54.0,45.0,...,54.0,79.0,38.0,37.0,42.0,43.0,63.0,50.0,56.0,42.0


In [18]:
epochs = 40
batch = 256
lr = 0.0003
adam = optimizers.Adam(lr)

In [19]:
X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_valid_series = X_valid.values.reshape((X_valid.shape[0], X_valid.shape[1], 1))
print('Train set shape', X_train_series.shape)
print('Validation set shape', X_valid_series.shape)

Train set shape (99790, 30, 1)
Validation set shape (66528, 30, 1)


### LSTM for Time Series Forecasting

In [20]:
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='relu', input_shape=(X_train_series.shape[1], X_train_series.shape[2])))
model_lstm.add(Dense(1))

model_lstm.compile(loss='mse', optimizer=adam)
model_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [21]:
lstm_history = model_lstm.fit(X_train_series, Y_train, validation_data=(X_valid_series, Y_valid), epochs=epochs, verbose=2)

Epoch 1/40
3119/3119 - 36s - loss: 2163.4373 - val_loss: 920.1475 - 36s/epoch - 12ms/step
Epoch 2/40
3119/3119 - 30s - loss: 895.6381 - val_loss: 883.3033 - 30s/epoch - 10ms/step
Epoch 3/40
3119/3119 - 29s - loss: 881.2687 - val_loss: 868.9422 - 29s/epoch - 9ms/step
Epoch 4/40
3119/3119 - 33s - loss: 919.9512 - val_loss: 904.7755 - 33s/epoch - 11ms/step
Epoch 5/40
3119/3119 - 33s - loss: 886.1106 - val_loss: 878.1739 - 33s/epoch - 11ms/step
Epoch 6/40
3119/3119 - 30s - loss: 877.4543 - val_loss: 868.1046 - 30s/epoch - 10ms/step
Epoch 7/40
3119/3119 - 34s - loss: 876.0655 - val_loss: 882.0546 - 34s/epoch - 11ms/step
Epoch 8/40
3119/3119 - 30s - loss: 871.1183 - val_loss: 856.4457 - 30s/epoch - 10ms/step
Epoch 9/40
3119/3119 - 32s - loss: 873.3083 - val_loss: 877.2628 - 32s/epoch - 10ms/step
Epoch 10/40
3119/3119 - 28s - loss: 861.0746 - val_loss: 848.9529 - 28s/epoch - 9ms/step
Epoch 11/40
3119/3119 - 33s - loss: 853.5309 - val_loss: 840.1127 - 33s/epoch - 10ms/step
Epoch 12/40
3119/311

In [ ]:
def mean_absolute_percentage_error( y, yhat ):
    return np.mean( np.abs( (y - yhat ) / y ) )

#### LSTM on train and validation

In [24]:
lstm_train_pred = model_lstm.predict(X_train_series)
lstm_valid_pred = model_lstm.predict(X_valid_series)

2079/2079 [==============================] - 8s 4ms/step
Train rmse: 33.2717832588677
Validation rmse: 33.16641812970795


In [40]:
# print('Train rmse:', np.sqrt(mean_squared_error(Y_train, lstm_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_valid, lstm_valid_pred)))

Validation rmse: 23.73441812970795


In [3]:
print('Validation mae:', np.sqrt(mean_absolute_error(Y_valid, lstm_valid_pred)))

Validation mae: 22.4583289


In [4]:
print('Validation mape:', np.sqrt(mean_absolute_percentage_error(Y_valid, lstm_valid_pred)))

Validation mape: 0.519390234


In [34]:
df_plot = train[train['date'] >= '2017-10-02']
df_plot['true'] = Y_train[0:45500]
df_plot['prediction'] = lstm_train_pred[0:45500]


In [37]:
# import seaborn             as sns
# import matplotlib.pyplot   as plt
# plt.subplot( 2, 2, 1 )
# sns.lineplot( x='date', y='true', data=df_plot, label='SALES' )
# sns.lineplot( x='date', y='prediction', data=df_plot, label='PREDICTIONS' )

In [31]:
X_test = train[train['date'] >= '2017-10-02']
y_test = X_test['sales']
dates = X_test['date']
len(y_test)

45500